In [1]:
using JuMP, Gurobi
using SparseArrays
using UnicodePlots

In [2]:
objective_function_pcpsp = Dict()
resource_constraint_ub_limits = Dict()
resource_constraint_lb_limits = Dict()
rc_coefficients = Dict()
open("../minelib_inputs/newman1.pcpsp") do file
    for ln in eachline(file)
        linea_array = split(ln)
        info = linea_array[1]
        if info == "NAME:"
            global nombre
            nombre = linea_array[2]
        elseif info == "NBLOCKS:"
            global n
            n = tryparse(Int64,linea_array[2])
        elseif info == "NPERIODS:"
            global T
            T = tryparse(Int64,linea_array[2])
        elseif info == "NDESTINATIONS:"
            global R
            R = tryparse(Int64,linea_array[2])
        elseif info == "NRESOURCE_SIDE_CONSTRAINTS:"
            global nresource_side_constraints
            nresource_side_constraints = tryparse(Int64,linea_array[2])
        elseif info == "NGENERAL_SIDE_CONSTRAINTS:"
            global ngeneral_side_constraints
            ngeneral_side_constraints = tryparse(Int64,linea_array[2])
        elseif info == "DISCOUNT_RATE:"
            global discount_rate
            discount_rate = tryparse(Float64,linea_array[2])
        elseif info == "RESOURCE_CONSTRAINT_LIMITS:"
            for r in 0:nresource_side_constraints-1 
                for t in 0:T-1 # reemplazar nresource side constraints 
                    linea = readline(file)
                    array = split(linea)
                    if array[3] == "L"
                        resource_constraint_ub_limits[r,t] = tryparse(Float64,array[end])
                        resource_constraint_lb_limits[r,t] = "-Infinity"
                    else
                        println("Este problema tiene cota inferiores.")
                        break
                    end
                end
            end
        elseif info == "OBJECTIVE_FUNCTION:"
            for b=0:n-1
                linea = readline(file)
                array = split(linea)
                for d=0:R-1
                    objective_function_pcpsp[b,d] = tryparse(Float64,array[d+1])
                end
            end
        elseif info == "RESOURCE_CONSTRAINT_COEFFICIENTS:"
            num_asig = 0
            for linea in  eachline(file)
                if linea == "EOF"
                    break
                end
                num_asig += 1
                array = split(linea)
                b = tryparse(Int64,array[1])
                d = tryparse(Int64,array[2])
                r = tryparse(Int64,array[3])
                rc_coefficients[b,r,d] = tryparse(Float64,array[4])
            end
        end
    end
end
asig = 0
iter = 0
for b=0:n-1
    for r=0:nresource_side_constraints-1
        for d=0:R-1
            iter += 1
            if .!((b,r,d) in keys(rc_coefficients))
                asig += 1
                #println((b,r,d) in keys(rc_coefficients))
                rc_coefficients[b,r,d] = 0
            end
        end
    end
end

# leer fn. objetivo de UPIT
obj_upit = Dict()
open("../minelib_inputs/newman1.upit") do file
    for i=1:4
        readline(file)
    end
    for ln in eachline(file)
        if .!(ln == "EOF")
            array = split(ln)
            block = tryparse(Int64,array[1])
            value = tryparse(Float64,array[2])
            obj_upit[block] = value
        end
    end
end
println("Data base name: $(nombre)")
println("NBLOCKS: $(n)")
println("NPERIODS: $(T)")
println("NDESTINATIONS: $(R)")
println("NRESOURCE_SIDE_CONSTRAINTS: $(nresource_side_constraints)" )
println("NGENERAL_SIDE_CONSTRAINTS: $(ngeneral_side_constraints)")
println("DISCOUNT_RATE: $(discount_rate)")

Data base name: Newman1
NBLOCKS: 1060
NPERIODS: 6
NDESTINATIONS: 2
NRESOURCE_SIDE_CONSTRAINTS: 2
NGENERAL_SIDE_CONSTRAINTS: 0
DISCOUNT_RATE: 0.08


In [6]:
# crear un diccionario con los vecinos (notar que solo asigno vecinos a los que efectivamente tienen)
vecinos = Dict()
open("../minelib_inputs/newman1.prec") do file
    for linea in eachline(file)
        linea_lista = split(linea)
        nvecinos = tryparse(Int64,linea_lista[2])
        a = tryparse(Int64,linea_lista[1])
        if nvecinos>0 # solo agrega vecinos si tiene!
            vecinos[a]=Set()
            for j=0:nvecinos-1 # reviasar que ocurre con los que no tienen vecinos!
                b= tryparse(Int64,linea_lista[j+3])
                push!(vecinos[a],b)
            end
        end
    end
end

In [7]:
length(vecinos)

981

In [9]:
# resolver UPIT
modelo_upit = Model(with_optimizer(Gurobi.Optimizer))
@variable(modelo_upit,x[i=0:n-1],Bin);
@expression(modelo_upit, obj, sum(x[i]*obj_upit[i] for i=0:n-1));
@objective(modelo_upit, Max, obj);

open("../minelib_inputs/newman1.prec") do file
    for ln in eachline(file)
        linea_array = split(ln)
        nvecinos = tryparse(Int64,linea_array[2])
        u = tryparse(Int64, linea_array[1])
        for j =0:nvecinos-1
            v = tryparse(Int64,linea_array[j+3])
            @constraint(modelo_upit,x[u] <= x[v])
        end
    end
end
status = optimize!(modelo_upit)
println("Optimal objective: ",objective_value(modelo_upit))

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Optimize a model with 3922 rows, 1060 columns and 7844 nonzeros
Variable types: 0 continuous, 1060 integer (1060 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 2e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 989307.55494
Found heuristic solution: objective 2.608690e+07
Presolve removed 3922 rows and 1060 columns
Presolve time: 0.03s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.04 seconds
Thread count was 1 (of 4 available processors)

Solution count 2: 2.60869e+07 989308 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.608689902597e+07, best bound 2.608689902597e+07, gap 0.0000%
Optimal objective: 2.60868990259699e7


In [10]:
# recuperar upit
blocks_upit = Set()
for b =0:n-1
    if value(x[b]) == 1
        push!(blocks_upit,b)
    end
end

In [11]:
# matrices sparse para pcpsp
# precedencia espacial
rows = Int64[]
cols_plus = Int64[]
cols_minus = Int64[]
data = Int64[]
row = 0
for t=0:T-1
    for b in keys(vecinos)
        for vecino in vecinos[b]
            row+=1
            push!(rows,row)
            push!(cols_plus,b + (R-1)*n + t*R*n + 1)
            push!(cols_minus,vecino + (R-1)*n + t*R*n + 1)
            push!(data,1)
        end
    end
end
AEplus = sparse(rows,cols_plus,data,row,n*R*T)
AEminus = sparse(rows,cols_minus,-data,row,n*R*T)
AE = AEplus + AEminus;

# matriz de precedencia en el destino
rows = Int64[]
cols_plus = Int64[]
cols_minus = Int64[]
data = Int8[]
row = 0
for t =0:T-1
    for d =0:R-2
        for b=0:n-1
            row +=1
            push!(rows,row)
            push!(cols_plus,b + d*n + t*R*n+1)
            push!(cols_minus,b + (d+1)*n + t*R*n+1)
            push!(data,1)
        end
    end
end
ADplus = sparse(rows,cols_plus,data,row,n*R*T)
ADminus = sparse(rows,cols_minus,-data,row,n*R*T)
AD = ADplus + ADminus;

# matriz de precedencia temporal 
rows = Int64[]
cols_plus = Int64[]
cols_minus = Int64[]
data = Int8[]
row = 0
for t =0:T-2
    for b =0:n-1 
        row+=1
        push!(rows,row) # revisar, quizas funciona de casualidad!!!
        push!(cols_plus,b+(R-1)*n+t*R*n+1)
        push!(data,1)
        push!(cols_minus,b+(t+1)*R*n+1)
    end
end
AT_plus =  sparse(rows,cols_plus,data,row,n*R*T)
AT_minus = sparse(rows,cols_minus,-data,row,n*R*T)
AT = AT_plus+AT_minus;

In [41]:
# recalcular coeficientes de la funcion objetivo para PCP_by
# Notar que LHS_sparse se puede hacer mejor eventualmente,
# puesto que se recorren muchos ceros en su definicion sparse
c = Dict()
for b in blocks_upit,d =0:R-1, t=0:T-1
    c[b,d,t] = (1.0/(1.0 + discount_rate))^t * objective_function_pcpsp[b,d]
end

c_hat = Dict()
for b in blocks_upit, d=0:R-1, t=0:T-1
    if t==0 && d==0
        c_hat[b,d,t] = -c[b,d,t]
    elseif d<=R-2 && t>=1
        c_hat[b,d,t] = c[b,d+1,t]
    elseif d==R-1 && t<=T-2
        c_hat[b,d,t] = c[b,0,t+1]
    else
        c_hat[b,d,t] = 0
    end
end
    
cx2 = Dict()
for b in blocks_upit, d=0:R-1,t=0:T-1
    if t==0
        cx2[b,d,t] = -c_hat[b,d,0]
    else
        cx2[b,d,t] = c[b,d,t]-c_hat[b,d,t]
    end
end

# lados derechos de las restricciones de precedencia
rhs_esp = spzeros(Int8,size(AE)[1]);
rhs_temp = spzeros(Int8,size(AT)[1]);
rhs_dest = spzeros(Int8,size(AD)[1]);

# LHS: a partir de la formula original
q = rc_coefficients
rows_1 = Int64[]
rows_2 = Int64[]
rows_3 = Int64[]
fila = 0
cols_1 = Int64[]
cols_2 = Int64[]
data_1 = Float64[]
data_2 = Float64[]
data_3 = Float64[]
cols_3 = Int64[]
cols_4 = Int64[]
for t=0:T-1
    for r=0:nresource_side_constraints-1
        fila+=1
        if t>0
            for b=0:n-1
                for d=0:R-1
                    if d>0
                        push!(rows_1,fila)
                        push!(cols_1,b+d*n+t*R*n+1)
                        push!(cols_2,b+(d-1)*n+t*R*n+1)
                        push!(data_1,q[b,r,d])
                    else
                        push!(rows_2,fila)
                        push!(rows_3,fila)
                        push!(cols_3,b+0*n+t*R*n+1)
                        push!(cols_4,b+(R-1)*n+(t-1)*R*n+1)
                        push!(data_2,q[b,r,0])
                        push!(data_3,-q[b,r,0])
                    end
                end
            end
        else
            for b=0:n-1
                for d =0:R-1
                    if d>0
                        push!(rows_1,fila)
                        push!(cols_1,b+d*n+1)
                        push!(cols_2,b+(d-1)*n+1)
                        push!(data_1,q[b,r,d])
                        
                    else
                        push!(rows_2,fila)
                        push!(cols_3,b+1)
                        push!(data_2,q[b,r,0])
                    end
                end
            end
        end
    end
end                                 
M1 = sparse(rows_1,cols_1, data_1,fila,n*R*T)
M2 = sparse(rows_1,cols_2,-data_1,fila,n*R*T)
M3 = sparse(rows_2,cols_3,data_2,fila,n*R*T)
M4 = sparse(rows_3,cols_4,data_3,fila,n*R*T)
LHS_sparse = M1+M2+M3+M4;

In [40]:
# recalcualr coeff de fn objetivo
c_array = Float64[]
for b in blocks_upit,d=0:R-1,t=0:T-1
    c_array[b+d*n+t*R*n] = cx2[b,d,t]
end

BoundsError: BoundsError: attempt to access 0-element Array{Float64,1} at index [306]

In [45]:
I = Int64[]
V = Float64[]
for b in blocks_upit, d = 0:R-1, t=0:T-1
    push!(I,b+n*d+t*R*n+1)
    push!(V,cx2[b,d,t])
end
cx = sparsevec(I,V);

In [25]:
# algunas funciones utiles
function check_opt_vect(sol, particion, ell, iteracion)
    for h=1:ell[iteracion-1]
        #arreglo = np.array([sol[iteracion][b].x for b in particion[h,iteracion-1]])
        #summ = np.matmul(arreglo,np.ones(arreglo.shape))
        summ = sum([sol[iteracion][b] for b in particion[h,iteracion-1]])
        if .!(isapprox(summ,0) || isapprox(summ,length(particion[h,iteracion-1])))
            return False
        end
    end
    return True
end

check_opt_vect (generic function with 1 method)

In [20]:
isapprox(2,2.000000004)

true

In [33]:
size(rhs_esp)

(23532,)

In [26]:
# BZ version esparse
problema_aux = Model(with_optimizer(Gurobi.Optimizer)) # modelo auxiliar
modelo_p2k = Model(with_optimizer(Gurobi.Optimizer))
@variable(problema_aux,0<=x[i=1:n*R*T] <=1)
@expression(problema_aux, cx_expr, sum(cx[i]*obj_upit[i] for i=0:n-1));
@constraint(problema_aux, precEsp, AE * x .<= rhs_esp);
@constraint(problema_aux, precDest, AD * x .<= rhs_dest);
@constraint(problema_aux, precTemp, AT * x .<= rhs_temp);
w = Dict()
eles = Dict()
eles[0] = 1
k = 1
mu = Dict()
mu[0] = Dict()
mu[0] = spzeros(Float64,1,T*nresource_side_constraints)
C = Dict()
C[1,0] = Set()
for b in blocks_upit, d=0:R-1, t=0:T-1
    push!(C[1,0],(b,d,t))
end
while true
    penalty = mu[k-1]*LHS_sparse
    if k>= && check_opt_vect(w,C,eles,k)
        println("Algoritmo termino: H^$(k-1) w[$(k)] = 0")
        println("Valor optimo de BZ: $(objective_value(modelo_p2k))")
        break
    end
end

Academic license - for non-commercial use only


In [35]:
?spzeros

search: spzeros set_optimizer



```
spzeros([type,]m[,n])
```

Create a sparse vector of length `m` or sparse matrix of size `m x n`. This sparse array will not contain any nonzero values. No storage will be allocated for nonzero values during construction. The type defaults to [`Float64`](@ref) if not specified.

# Examples

```jldoctest
julia> spzeros(3, 3)
3×3 SparseMatrixCSC{Float64,Int64} with 0 stored entries

julia> spzeros(Float32, 4)
4-element SparseVector{Float32,Int64} with 0 stored entries
```


In [36]:
mu[0] = Dict()
mu[0] = spzeros(Float64,1,T*nresource_side_constraints)

1×12 SparseMatrixCSC{Float64,Int64} with 0 stored entries

In [ ]:
size(x)
size(AE)
reshape(AE,(23532,n*R*T))*reshape(x,(n*R*T,1))

In [22]:
model = Model(with_optimizer(Gurobi.Optimizer))
@variable(model, x[i=1:2])
A = [1 2; 3 4]
Asp = sparse(A)
b = zeros(n)
@constraint(model, con, Asp * x .<= b)

Academic license - for non-commercial use only


2-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1}:
 x[1] + 2 x[2] <= 5.0  
 3 x[1] + 4 x[2] <= 6.0

In [27]:
typeof(x),size(x)

(Array{VariableRef,1}, (12720,))

In [28]:
typeof(AE), size(AE)

(SparseMatrixCSC{Int64,Int64}, (23532, 12720))

In [14]:
size(AE)

(23532, 12720)

In [15]:
typeof(AE)

SparseMatrixCSC{Int64,Int64}

In [20]:
size(rhs_prec)

(12720,)

In [16]:
typeof(Asp)

SparseMatrixCSC{Int64,Int64}

In [105]:
toy = spzeros(Int8,n*R*T)
size(toy)

(12720,)

In [41]:
length(data)

23532

In [275]:
status = optimize!(m)
println("Optimal objective: ",objective_value(m),
	". x = ", value(x), " y = ", value(y))

Academic license - for non-commercial use only
Optimize a model with 2 rows, 2 columns and 4 nonzeros
Coefficient statistics:
  Matrix range     [2e+01, 5e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e+00, 5e+01]
  RHS range        [2e+03, 2e+03]
Presolve removed 2 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.0000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  5.000000000e+01
Optimal objective: 50.0. x = 5.0 y = 45.0


In [267]:
?objective_value

search: objective_value



```
objective_value(model::Model)
```

Return the objective value after a call to `optimize!(model)`.


In [269]:
?Model

search: Model ModelMode model_type model_name model_sense model_string



```
Model
```

A mathematical model of an optimization problem.

---

```
Model(; caching_mode::MOIU.CachingOptimizerMode=MOIU.AUTOMATIC,
        bridge_constraints::Bool=true)
```

Return a new JuMP model without any optimizer; the model is stored the model in a cache. The mode of the `CachingOptimizer` storing this cache is `caching_mode`. The optimizer can be set later in the [`optimize!`](@ref) call. If `bridge_constraints` is true, constraints that are not supported by the optimizer are automatically bridged to equivalent supported constraints when an appropriate transformation is defined in the `MathOptInterface.Bridges` module or is defined in another module and is explicitely added.

---

```
Model(optimizer_factory::OptimizerFactory;
      caching_mode::MOIU.CachingOptimizerMode=MOIU.AUTOMATIC,
      bridge_constraints::Bool=true)
```

Return a new JuMP model using the optimizer factory `optimizer_factory` to create the optimizer. The optimizer factory can be created by the [`with_optimizer`](@ref) function.

## Examples

The following creates a model using the optimizer `IpoptOptimizer(print_level=0)`:

```julia
model = Model(with_optimizer(Ipopt.Optimizer, print_level=0))
```


In [247]:
ks = keys(rc_coefficients);
typeof(ks)

Base.KeySet{Any,Dict{Any,Any}}

In [253]:
collect(rc_coefficients)

3180-element Array{Pair{Any,Any},1}:
  (798, 1, 1) => 5664.0 
  (129, 0, 1) => 5664.0 
  (103, 1, 1) => 5644.81
  (618, 0, 0) => 5664.0 
  (928, 1, 1) => 5184.0 
 (1014, 0, 0) => 5664.0 
  (376, 1, 1) => 2994.93
  (421, 0, 0) => 5664.0 
  (637, 1, 1) => 5664.0 
  (533, 1, 1) => 5664.0 
  (718, 0, 0) => 5664.0 
  (119, 0, 1) => 5664.0 
  (811, 1, 1) => 5664.0 
              ⋮         
  (124, 1, 1) => 5664.0 
  (691, 0, 0) => 5664.0 
  (335, 0, 1) => 2613.03
  (745, 1, 1) => 5664.0 
  (252, 0, 1) => 5664.0 
  (430, 0, 0) => 5020.3 
   (28, 1, 1) => 1868.55
  (468, 0, 1) => 5664.0 
  (597, 0, 0) => 5664.0 
  (882, 0, 1) => 5664.0 
  (171, 0, 1) => 5664.0 
 (1049, 1, 1) => 5664.0 

In [148]:
n*nresource_side_constraints*R

4240

In [232]:
rc_coefficients

Dict{Any,Any} with 6361 entries:
  (798, 1, 1)  => 5664.0
  (331, 1, 2)  => 0
  (934, 1, 2)  => 0
  (540, 2, 2)  => 0
  (129, 0, 1)  => 5664.0
  (103, 1, 1)  => 5644.81
  (618, 0, 0)  => 5664.0
  (928, 1, 1)  => 5184.0
  (79, 2, 1)   => 0
  (82, 2, 1)   => 0
  (1014, 0, 0) => 5664.0
  (230, 1, 2)  => 0
  (547, 2, 1)  => 0
  (376, 1, 1)  => 2994.93
  (421, 0, 0)  => 5664.0
  (90, 1, 2)   => 0
  (637, 1, 1)  => 5664.0
  (533, 1, 1)  => 5664.0
  (718, 0, 0)  => 5664.0
  (14, 2, 2)   => 0
  (301, 2, 2)  => 0
  (623, 2, 1)  => 0
  (184, 2, 1)  => 0
  (186, 2, 1)  => 0
  (240, 1, 2)  => 0
  ⋮            => ⋮

In [217]:
if .!(1 in keys(Dict(0=>1,2=>2,3=>3)))
    println("hola")
end

hola


In [214]:
.!(1 in keys(Dict(0=>1,2=>2,3=>3)))

true

In [223]:
keys(rc_coefficients)

Base.KeySet for a Dict{Any,Any} with 6361 entries. Keys:
  (798, 1, 1)
  (331, 1, 2)
  (934, 1, 2)
  (540, 2, 2)
  (129, 0, 1)
  (103, 1, 1)
  (618, 0, 0)
  (928, 1, 1)
  (79, 2, 1)
  (82, 2, 1)
  (1014, 0, 0)
  (230, 1, 2)
  (547, 2, 1)
  (376, 1, 1)
  (421, 0, 0)
  (90, 1, 2)
  (637, 1, 1)
  (533, 1, 1)
  (718, 0, 0)
  (14, 2, 2)
  (301, 2, 2)
  (623, 2, 1)
  (184, 2, 1)
  (186, 2, 1)
  (240, 1, 2)
  ⋮

In [151]:
length(objective_function_pcpsp)

2120

In [161]:
length(rc_coefficients)

387

In [118]:
resource_constraint_ub_limits

Dict{Any,Any} with 12 entries:
  (1, 6) => 2.0e6
  (2, 3) => 1.1e6
  (2, 6) => 1.1e6
  (2, 2) => 1.1e6
  (2, 1) => 1.1e6
  (2, 5) => 1.1e6
  (1, 4) => 2.0e6
  (1, 3) => 2.0e6
  (2, 4) => 1.1e6
  (1, 1) => 2.0e6
  (1, 2) => 2.0e6
  (1, 5) => 2.0e6

In [116]:
resource_constraint_lb_limits

Dict{Any,Any} with 12 entries:
  (1, 6) => "-Infinity"
  (2, 3) => "-Infinity"
  (2, 6) => "-Infinity"
  (2, 2) => "-Infinity"
  (2, 1) => "-Infinity"
  (2, 5) => "-Infinity"
  (1, 4) => "-Infinity"
  (1, 3) => "-Infinity"
  (2, 4) => "-Infinity"
  (1, 1) => "-Infinity"
  (1, 2) => "-Infinity"
  (1, 5) => "-Infinity"

In [123]:
objective_function_pcpsp

Dict{Any,Any} with 0 entries

In [66]:
f = open("../minelib_inputs/newman1.pcpsp")
linea = readline(f)
linea_splitted = split(linea)
valor = linea_splitted[1]=="NAME:"
close(f)

In [67]:
typeof(linea_splitted[1])

SubString{String}

In [69]:
if linea_splitted[1] == "NAME:"
    print("hola")
end

hola

In [92]:
for i=1:10
    tt = 10
    for tiempo =1:tt
        println(tiempo)
    end
end

1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10
